# 通过多级注意力CNNs进行关系分类
王琳林1，朱曹友1,Gerard de Melo2, Zhiyuan Liu3
1清华大学跨学科信息科学研究所，北京2罗格斯大学计算机科学系，皮斯卡塔韦，美国新泽西州
3清华大学信息科学与技术国家实验室智能技术与系统国家重点实验室，
清华大学计算机科学与技术学院，北京，中国
{ll-wang13, cao-z13} @mails.tsinghua.edu.cn, gdm@demelo.org
 
# 摘要
关系分类是众多从文本中挖掘结构化事实的信息抽取系统中的一个重要组成部分。为了更好地识别异构环境中的模式，我们提出了一种新的卷积神经网络结构。这种体系结构支持从特定于任务的标记数据进行端到端学习，而不需要外部知识，比如显式依赖结构。实验表明，我们的模型超越了以前最先进的方法，包括那些依赖于更丰富的先验知识形式的方法。

# 1介绍

关系分类是识别文本中两个名词性之间的语义关系。它是自然语言处理系统的一个重要组成部分，需要从文本中挖掘出与之相关的事实，例如用于各种信息提取应用以及问题转换和知识库完成(Tandon et al.， 2011;陈等，2015)。例如，给定示例输入“碳酸饮料和肉类会导致心脏病和糖尿病。”

带注释的目标实体提到e1 = " drinks "和e2 = " diabetes "，目标是自动识别这句话表达了e1和e2之间的因果关系，对此我们使用因果符号(e1,e2)。准确的关系分类有助于准确的句子解释、语篇处理和更高层次的NLP任务(Hendrickx et al.， 2010)。因此，在过去的几十年里，关系分类引起了研究者的广泛关注(Zhang, 2004;钱等，2009;Rink and Harabagiu, 2010)。

在上面给出的例子中，动词corre-sponds非常接近期望的目标关系。然而，在野外，我们会遇到很多不同的方式来表达同一种关系。这种具有挑战性的可变性可以是词汇的、句法的，甚至是语用的。有效的解决方案需要能够考虑有用的语义和句法特征，不仅要考虑目标实体在词汇级别上的含义，还要考虑它们的即时上下文和整个句子结构。

因此，提出了许多基于特征和内核的方法也就不足为奇了，其中许多方法都依赖于成熟的NLP堆栈，包括词性标注、形态学分析、去依赖性解析，偶尔也会进行语义分析，以及利用知识资源来捕获词汇和语义特征(Kambhatla, 2004;周等，2005;Suchanek等，2006;钱等，2008;穆尼和布内斯库，2005;布内斯库和穆尼，2005)。近年来，我们已经看到了一种向深层架构发展的趋势，这种架构能够学习相关的表示和特性，而不需要大量的手工特性工程或使用外部资源。已有许多卷积神经网络(CNN)、递归神经网络(RNN)等神经结构被提出用于关系分类(Zeng et al.， 2014;dos Santos等，2015;徐等，2015b)。尽管如此，这些模型常常不能识别关键线索，其中许多仍然需要外部依赖解析器。

我们提出了一个新颖的CNN架构，它弥补了之前ap-proaches的一些缺点。我们的主要贡献如下:

 1. 我们的CNN架构依赖于一种新颖的多层注意力机制来捕获实体特定的注意力(输入级的主要注意力，相对于目标关系)和关系特定的集中注意力(相对于目标关系的次要注意力)。这使得它能够检测到更细微的线索，尽管输入句子的结构是异构的，这使它能够自动地学习哪些部分与给定的分类相关。
 
 2.	提出了一种新的基于双边距的目标函数，证明了它优于标准损失函数。
 
 3.	在SemEval 2010 Task 8数据集上，我们获得了最新的关系分类结果，F1得分为88.0%，优于依赖于显著丰富的先验知识的方法。

# 2.相关工作
除了少数无监督聚类meth-ods (Hasegawa et al.， 2004;Chen et al.， 2005)，大多数关系分类的工作都是在监督下进行的，通常被转换为一个标准的多类或多标签分类任务。传统的基于特征的方法依赖于从显式语言预处理步骤的输出计算出的一组特征(Kambhatla, 2004;周等，2005;Boschee等，2005;Suchanek等，2006;Chan和Roth, 2010;而基于核的方法使用卷积树核(Qian et al.， 2008)、子序列核(Mooney and Bunescu, 2005)或依赖树核(Bunescu and Mooney, 2005)。因此，这些方法都依赖于精心设计的特性，这些特性通常是在反复试验的基础上选择的，或者依赖于精心设计的内核，而这些内核又常常来自其他预先训练好的NLP工具或词汇和语义资源。虽然这些方法可以从外部NLP工具中获益，以发现句子的dis-crete结构，但是语法分析是容易出错的，依赖于它的成功也可能阻碍性能(Bach和Badaskar, 2007)。进一步的缺点包括它们对不可见单词的词法元化能力有限，以及它们在应用于新领域、类型或语言时缺乏健壮性。

近年来，深度神经网络已显示出良好的应用前景。Socher等人(2012)的递归矩阵-向量模型(Recursive Matrix-Vector Model, MV-RNN)试图利用句法树来捕捉句子语义的成分方面。Zeng等(2014)提出了一种深度卷积神经网络，具有软最大分类、提取词汇和句子层次特征。然而，这些方法仍然依赖于词汇资源和NLP工具包的附加特性。Yu等(2014)提出了基于因子的成分嵌入模型，该模型使用句法深度树和句子级嵌入。除了dos Santos等(2015)提出了类嵌入矩阵的CNN (CR-CNN)排名模型外，Miwa和Bansal(2016)同样发现，由于网络存档结构中捕捉到的语言结构有限，基于lstm的RNNs比使用CNNs的模型表现得更好。为了解决这个问题，已经提出了一些更精细的变体，包括双向LSTMs (Zhang et al.， 2015)、深度递归神经网络(Xu et al.， 2016)和双向树形LSTM-RNNs (Miwa和Bansal, 2016)。最近的几项工作也重新引入了一种基于依赖树的设计，例如，在句法树上运行的RNNs(桥本等，2013)，基于最短深度路径的CNNs (Xu等，2015a)，以及SDP-LSTM模型(Xu等，2015b)。最后，Nguyen和Grishman(2015)对CNNs和RNNs进行培训，并使用投票、叠加或对数线性模型对它们的输出进行不同的聚合(Nguyen和Grishman, 2015)。尽管这些最新的模式取得了可靠的结果，但在理想情况下，我们希望有一个简单而有效的体系结构，不需要重新查询依赖项解析或培训多个模式。我们在第4节的实验证明，我们确实可以做到这一点，同时也获得了F1分数方面的实质性改善。

# 3.提出的模型
给定一个带有一对实体标记的句子S提到e1和e2(正如我们在第1节的例子中所述)，关系分类的任务是识别一组候选关系类型中e1和e2之间的语义关系(Hendrickx et al.， 2010)。由于唯一的输入是带有两个标记提及的原始感觉，因此要获得准确预测所需的所有词汇、语义和句法线索并非易事。为此，我们提出了一种基于多级注意的卷积神经网络模型。图1给出了我们架构的示意图。首先对输入语句进行编码使用字向量表示，利用上下文和位置编码，以便更好地捕获这个词的顺序。一个主要的注意力机制，采用基于对角矩阵的方法来捕获与目标词相关的词tities。得到的输出矩阵为1
应用卷积运算来捕获上下文信息，如相关的n-gram，max-pooling紧随其后。一个次要的关注池化层用于确定最有用的卷积特征进行关系分类基于注意力集中矩阵的输出。本节的其余部分将提供更多信息关于此架构的详细信息。表1提供了我们将使用的符号概述。的最终的输出由一个新的目标函数给出，下面描述。